In [244]:
import csv
import re
import sys
import time
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn import metrics, tree
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score, precision_score,
                             recall_score)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.feature_selection import chi2, SelectKBest

In [245]:
df = pd.read_csv('dataset.tsv', sep='\t', quoting=csv.QUOTE_NONE, dtype=str,
                 header=None, names=["instance", "text", "id", "sentiment", "is_sarcastic"])
df = shuffle(df)

In [246]:
text_data = np.array([])
# Read tweets
for text in df.text:
    text_data = np.append(text_data, text)

In [247]:
""" Functions for text pre-processing """


def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", " ", sample)


def remove_punctuation(sample):
    """Remove punctuations from a sample string"""
    punctuations = r'''$!"&'()*+,-./:;<=>?[\]^`{|}~'''
    no_punct = ""
    for char in sample:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

def myTokenizer(sample):
    """Customized tokenizer"""
    ################################## 1. Remove numbers
    ################################## 2. Remove auspoll thingy
    ################################## 3. Remove starts with au
    new_words = []
    words = sample.split(' ')
    new_words = [word for word in words if len(word) >= 2 and not word.isdigit() and not word.startswith('#aus') and not word.startswith('au')]
    return new_words

def remove_stopwords_NLTK(sample):
    """Remove stopwords using NLTK"""
    stopWords = set(stopwords.words('english'))
    words = myTokenizer(sample)
    filteredText = ""
    for word in words:
        if word not in stopWords:
            filteredText = filteredText + word + " "
    return filteredText.rstrip()


def porter_stem(sample):
    """Stemming"""
    words = myTokenizer(sample)
    ps = PorterStemmer()
    stemmed_text = ""
    for word in words:
        stemmed_text = stemmed_text + ps.stem(word) + " "
    return stemmed_text.rstrip()


def myPreprocessor(sample):
    """Customized preprocessor"""
    sample = remove_URL(sample)
    sample = sample.lower()
    sample = remove_stopwords_NLTK(sample)
    sample = remove_punctuation(sample)
    sample = porter_stem(sample)
    return sample


In [248]:
# try to use sklearn stop_words later
count = CountVectorizer(preprocessor=myPreprocessor, tokenizer=myTokenizer, max_features=1000)
bag_of_words = count.fit_transform(text_data)
print(count.get_feature_names())
size = len(count.vocabulary_)
print(len(count.vocabulary_))

['#4corner', '#7new', '#abc730', '#abcnews24', '#afpraid', '#agchatoz', '#alp', '#asylumseek', '#betterfutur', '#brexit', '#budget2016', '#canberra', '#cfa', '#cfmeu', '#chafta', '#childcar', '#cleanenergi', '#climat', '#climatechang', '#climatesci', '#coal', '#corrupt', '#csg', '#csiro', '#csirocut', '#dutton', '#educ', '#election2016', '#environ', '#et', '#faketradi', '#fraudband', '#gonski', '#greatbarrierreef', '#green', '#greens16', '#humanright', '#icac', '#inequ', '#insid', '#labor', '#laborlaunch', '#latelin', '#leadersdeb', '#liber', '#libral', '#lnp', '#lnpfail', '#malcolm', '#malwar', '#marriageequ', '#medicar', '#nauru', '#nbn', '#nbnco', '#nbngate', '#ndi', '#negativegear', '#npc', '#nswpol', '#panamapap', '#parakeelia', '#penaltyr', '#peoplesforum', '#pmlive', '#polita', '#putlnplast', '#qanda', '#qldpol', '#refuge', '#renew', '#safeschool', '#savemedicar', '#scomo', '#spendomet', '#springst', '#ssm', '#stopstateterror', '#taxrort', '#tennew', '#thedrum', '#turnbul', '1%'

In [249]:
X = bag_of_words.toarray()
# creating target classes
Y = np.array([])
for text in df.id:
    Y = np.append(Y, text)
# First 1500 for training set, last 500 for test set
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.25, shuffle=False)

In [250]:
clf = MultinomialNB()
# clf = BernoulliNB()
model = clf.fit(X_train, y_train)
text_clf_red = Pipeline([('vect', CountVectorizer(preprocessor=myPreprocessor, tokenizer=myTokenizer)), 
                       ('reducer', SelectKBest(chi2, k=800)),
                       ('clf', MultinomialNB())
                       ])
model_new = text_clf_red.fit(text_data[:1500],Y[:1500])

In [251]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       10000       0.52      0.73      0.61        56
       10001       0.41      0.35      0.38        31
       10002       0.46      0.35      0.40        31
       10003       0.38      0.59      0.46        79
       10004       0.00      0.00      0.00         6
       10005       0.52      0.64      0.57        47
       10006       0.35      0.30      0.32        53
       10007       0.00      0.00      0.00         1
       10008       0.60      0.64      0.62        47
       10009       0.50      0.33      0.40         3
       10010       0.33      0.08      0.13        12
       10011       0.00      0.00      0.00         2
       10012       0.00      0.00      0.00         8
       10013       0.55      0.53      0.54        32
       10014       0.00      0.00      0.00         8
       10015       0.63      0.59      0.61        32
       10016       0.00      0.00      0.00        15
       10017       0.00    

C:\Users\Doodey\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [252]:
y_pred = model.predict(X_train)
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

       10000       0.82      0.86      0.84       188
       10001       0.76      0.69      0.72       109
       10002       0.70      0.82      0.76        99
       10003       0.65      0.77      0.71       279
       10004       1.00      0.18      0.31        11
       10005       0.70      0.85      0.77       147
       10006       0.78      0.73      0.75       136
       10007       0.00      0.00      0.00         6
       10008       0.79      0.86      0.82       116
       10009       0.88      0.54      0.67        13
       10010       0.69      0.50      0.58        44
       10011       1.00      0.18      0.31        11
       10012       0.80      0.47      0.59        17
       10013       0.86      0.86      0.86        72
       10014       1.00      0.38      0.55        21
       10015       0.89      0.93      0.91        87
       10016       0.73      0.43      0.54        44
       10017       0.81    

C:\Users\Doodey\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [253]:
y_pred = model_new.predict(text_data[1500:])
print(classification_report(Y[1500:], y_pred))

              precision    recall  f1-score   support

       10000       0.56      0.57      0.57        56
       10001       0.50      0.23      0.31        31
       10002       0.54      0.48      0.51        31
       10003       0.30      0.75      0.42        79
       10004       0.00      0.00      0.00         6
       10005       0.55      0.64      0.59        47
       10006       0.59      0.30      0.40        53
       10007       0.00      0.00      0.00         1
       10008       0.58      0.64      0.61        47
       10009       0.00      0.00      0.00         3
       10010       0.50      0.08      0.14        12
       10011       0.00      0.00      0.00         2
       10012       0.00      0.00      0.00         8
       10013       0.75      0.56      0.64        32
       10014       1.00      0.12      0.22         8
       10015       0.86      0.59      0.70        32
       10016       0.00      0.00      0.00        15
       10017       0.50    

In [254]:
y_pred = model_new.predict(text_data[:1500])
print(classification_report(Y[:1500], y_pred))

              precision    recall  f1-score   support

       10000       0.76      0.76      0.76       188
       10001       0.68      0.33      0.44       109
       10002       0.66      0.71      0.68        99
       10003       0.48      0.82      0.61       279
       10004       1.00      0.45      0.62        11
       10005       0.61      0.71      0.66       147
       10006       0.82      0.49      0.61       136
       10007       1.00      0.83      0.91         6
       10008       0.73      0.82      0.77       116
       10009       1.00      0.31      0.47        13
       10010       1.00      0.34      0.51        44
       10011       1.00      0.45      0.62        11
       10012       0.83      0.59      0.69        17
       10013       0.87      0.74      0.80        72
       10014       1.00      0.81      0.89        21
       10015       0.80      0.89      0.84        87
       10016       1.00      0.32      0.48        44
       10017       0.91    